In [1]:
%config IPCompleter.greedy=True

In [1]:
import collections
from datetime import date
from dateutil.relativedelta import *
from faker import Faker
from collections import namedtuple, Counter
from time import perf_counter
from functools import wraps
from random import shuffle, seed, randint, uniform, choice, sample
from faker.providers.company import Provider

In [3]:
Faker.seed(1)
fake = Faker()

In [2]:
def get_fake_profiles(type, count):
    fake_profiles = []
    if type == 'namedtuple':
        profile = namedtuple('Profile', ['job', 'company', 'ssn', 'residence', 'current_location',
                                         'blood_group', 'website', 'username', 'name', 'sex', 'address', 'mail', 'birthdate'])
        for _ in range(count):
            fake_profiles.append(profile(**fake.profile()))
    elif type == 'dict':
        for _ in range(count):
            fake_profiles.append(fake.profile())
    return fake_profiles


def get_largest_blood_group(collection, type):
    bg_counts = None
    if type == 'namedtuple':
        bg_counts = Counter(p.blood_group for p in collection)
    elif type == 'dict':
        bg_counts = Counter(p['blood_group'] for p in collection)
    return bg_counts.most_common()[0]


def get_oldest_birthdate(collection, type):
    oldest_birthdate = None
    if type == 'namedtuple':
        oldest_birthdate = min(
            collection, key=lambda a: a.birthdate).birthdate
    elif type == 'dict':
        oldest_birthdate = min(
            collection, key=lambda a: a['birthdate'])['birthdate']
    return oldest_birthdate


def get_average_age(collection, type):
    if type == 'namedtuple':
        return sum([relativedelta(date.today(), a.birthdate).years for a in collection])/len(collection)
    elif type == 'dict':
        return sum([relativedelta(date.today(), a['birthdate']).years for a in collection])/len(collection)

In [4]:
NO_OF_PROFILES = 10000
fake_profiles = get_fake_profiles('namedtuple', NO_OF_PROFILES)
fake_profiles_dict = get_fake_profiles('dict', NO_OF_PROFILES)

In [5]:
def get_average_time(n, func, *args, **kwargs):
    start = perf_counter()
    for _ in range(n):
        value = func(*args, **kwargs)
    avg_run_time = (perf_counter() - start)/n
    return avg_run_time


N = 1000
for f in [get_largest_blood_group, get_oldest_birthdate, get_average_age]:
    print('testing:', f.__name__)
    t1 = get_average_time(N, f, fake_profiles, 'namedtuple')
    t2 = get_average_time(N, f, fake_profiles_dict, 'dict')

    if t1 < t2:
        print('dict is faster')
    else:
        print('named tuple is faster')

testing: get_largest_blood_group
dict is faster
testing: get_oldest_birthdate
dict is faster
testing: get_average_age
named tuple is faster


In [6]:
company_stck = namedtuple('Company', ['name', 'symbol', 'open', 'high', 'low', 'close','weight'])

In [7]:
def get_company_stock(volatility,direction):              
    name = fake.unique.company()
    symbol = name[0:3].upper()+name[-3:].upper()
    open = round( uniform(10,1000),2)
    if direction == '-ve':
        volatility *= -1
    close =  round(open + (open * volatility * 0.2),2)
    high = close = round(open + (open * volatility * 0.5),2)
    low = close = round(open - (open * volatility * 0.5),2)
    return name, symbol, open, high,low, close

In [10]:
weights = sample(range(1,200),100)

In [11]:
stock_list = [company_stck(*get_company_stock(volatility = uniform(.02, .2),direction=choice(['+ve','-ve'])),w) for w in weights]


In [12]:
stock_list

[Company(name='Weaver Ltd', symbol='WEALTD', open=466.68, high=450.03, low=483.33, close=483.33, weight=50),
 Company(name='James, Graves and Phillips', symbol='JAMIPS', open=863.24, high=813.11, low=913.37, close=913.37, weight=90),
 Company(name='Johnson Group', symbol='JOHOUP', open=539.46, high=585.04, low=493.88, close=493.88, weight=128),
 Company(name='Carter Group', symbol='CAROUP', open=620.53, high=578.37, low=662.69, close=662.69, weight=105),
 Company(name='Lang Inc', symbol='LANINC', open=545.83, high=512.6, low=579.06, close=579.06, weight=130),
 Company(name='Price-Howard', symbol='PRIARD', open=663.04, high=696.39, low=629.69, close=629.69, weight=176),
 Company(name='Gomez Inc', symbol='GOMINC', open=934.91, high=879.03, low=990.79, close=990.79, weight=27),
 Company(name='Walsh, Anderson and Hill', symbol='WALILL', open=466.62, high=459.87, low=473.37, close=473.37, weight=20),
 Company(name='Miller Group', symbol='MILOUP', open=436.44, high=466.89, low=405.99, close=

In [13]:
stock_list.sort(key = lambda a: a.weight)

In [14]:
stock_market_open = sum([a.open for a in stock_list[:50]])
stock_market_high = sum([a.high for a in stock_list[:50]])
stock_market_close = sum([a.close for a in stock_list[:50]])

In [15]:
print('stock market opened at:',stock_market_open)
print('stock market highest value was:',stock_market_high)
print('stock market closed at:',stock_market_close)

stock market opened at: 26183.13
stock market highest value was: 26158.820000000003
stock market closed at: 26207.440000000006
